### Urban Classified Script for Sentinel-2

<b>General description of the script:</b>
<br>The script uses the NDWI, NDVI, BarrenSoil and B11 to differntiate between water, built up areas, barren areas and vegetated areas. In the script, water is colored blue, vegetation green, built up areas white, barren soil brown and all other pixels dark green. All pixels with NDWI values greater than 0.2 are considered to be water; all other pixels with B11 values greater than 0.8 OR NDVI values lower than 0.1 to be built up; all other pixels with NDVI values greater than 0.2 to be vegetation (returnint NDVI in the green channel) and all other pixels to be barren soil or else (returnint BareSoil index in the red channel and dark green in the green channel.). The script does a good job (although not perfect) at separating barren soil from buildings, which is often an issuewith urban scripts. It is thus most valuable in arid regions, where most other visualizations fail to distinguish buildings from the surrounding sand or soil.

Original JavaScript from:
https://custom-scripts.sentinel-hub.com/sentinel-2/urban_classified/

In [ ]:
//Urban Classified Script
//by Monja Sebela

var NDWI=index(B03,B08); 
var NDVI=index(B08, B04);
var BareSoil=2.5 *((B11 + B04)-(B08 + B02))/((B11 + B04)+(B08 + B02));
 
if (NDWI > 0.2) {
 return [0, 0.5, 1]
}
if((B11>0.8)||(NDVI<0.1)){
  return[1,1,1]
}
if (NDVI>0.2){
  return [0, 0.3*NDVI, 0]
}
else {
 return [BareSoil, 0.2, 0]
}

Python Script for Urban Classified:

In [ ]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling

def resample_to_match(src, reference):
    """Resample a raster to match a reference raster."""
    data = src.read(1)
    dst_shape = (reference.height, reference.width)
    dst_array = np.empty(dst_shape, dtype=np.float32)

    reproject(
        source=data,
        destination=dst_array,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=reference.transform,
        dst_crs=reference.crs,
        resampling=Resampling.bilinear
    )

    return dst_array

def urban_classified(ndwi_path, ndvi_path, bsi_path, b11_path, output_path):
    with rasterio.open(ndwi_path) as ndwi_src, \
         rasterio.open(ndvi_path) as ndvi_src, \
         rasterio.open(bsi_path) as bsi_src, \
         rasterio.open(b11_path) as b11_src:

        # Read NDWI as base and resample others
        ndwi = ndwi_src.read(1)
        ndvi = resample_to_match(ndvi_src, ndwi_src)
        bsi = resample_to_match(bsi_src, ndwi_src)
        b11 = resample_to_match(b11_src, ndwi_src)

        # Clean nodata values
        ndwi[ndwi < -1e10] = 0
        ndvi[ndvi < -1e10] = 0
        bsi[bsi < -1e10] = 0
        b11[b11 < -1e10] = 0

        height, width = ndwi.shape
        classified = np.zeros((3, height, width), dtype=np.float32)

        # Updated classification thresholds
        water_mask = ndwi > 0.2
        builtup_mask = (~water_mask) & (b11 > 0.9) & (ndvi < 0.05)
        vegetation_mask = (~water_mask) & (~builtup_mask) & (ndvi > 0.2)
        barren_mask = (~water_mask) & (~builtup_mask) & (~vegetation_mask)

        # Water - Bright Blue
        classified[0][water_mask] = 0
        classified[1][water_mask] = 0.3
        classified[2][water_mask] = 1

        # Built-up - Light Gray
        classified[0][builtup_mask] = 0.8
        classified[1][builtup_mask] = 0.8
        classified[2][builtup_mask] = 0.8

        # Vegetation - Brighter Green
        classified[0][vegetation_mask] = 0
        classified[1][vegetation_mask] = 0.6 * ndvi[vegetation_mask]
        classified[2][vegetation_mask] = 0

        # Barren Soil - Reddish Brown / Orange
        classified[0][barren_mask] = bsi[barren_mask]
        classified[1][barren_mask] = 0.3
        classified[2][barren_mask] = 0

        # Clip values to [0, 1] and convert to uint8
        np.clip(classified, 0, 1, out=classified)
        classified_uint8 = (classified * 255).astype(np.uint8)

        # Save output
        profile = ndwi_src.profile
        profile.update({
            'count': 3,
            'dtype': 'uint8',
            'compress': 'lzw',
            'nodata': 0
        })

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(classified_uint8)

    print(f"✅ Classified land cover image saved to: {output_path}")



# Function Usage
urban_classified(
    ndwi_path='/Users/marzi\OneDrive - University of New Mexico/EPS 522 - Drones/Assignments/Assignment 6 - Intro Satellite Remote Sensing/S2_VirtualRasterLayers/VirtualRaster_S2_3Oct2023_NDWILayer.tif',
    ndvi_path='/Users/marzi\OneDrive - University of New Mexico/EPS 522 - Drones/Assignments/Assignment 6 - Intro Satellite Remote Sensing/S2_VirtualRasterLayers/VirtualRaster_S2_3Oct2023_NDVILayer.tif',
    bsi_path='/Users/marzi\OneDrive - University of New Mexico/EPS 522 - Drones/Assignments/Assignment 6 - Intro Satellite Remote Sensing/S2_VirtualRasterLayers/VirtualRaster_S2_3Oct2023_BSILayer.tif',
    b11_path='/Users/marzi\OneDrive - University of New Mexico/EPS 522 - Drones/Assignments/Assignment 6 - Intro Satellite Remote Sensing/Sentinel-2 Data/2023-10-03-00_00_2023-10-03-23_59_Sentinel-2_L2A_B11_(Raw).tiff',
    output_path='/Users/marzi\OneDrive - University of New Mexico/EPS 522 - Drones/Assignments/Assignment 6 - Intro Satellite Remote Sensing/S2_VirtualRasterLayers/Urban_Classified.tif'
)

✅ Classified land cover image saved to: /Users/marzi\OneDrive - University of New Mexico/EPS 522 - Drones/Assignments/Assignment 6 - Intro Satellite Remote Sensing/S2_VirtualRasterLayers/Urban_Classified.tif
